In [1]:
import json
import pandas as pd
 
coords_path = 'data/2022-12-17 15:52:24.006277-coords.json'
data_path = 'data/2022-12-17 14:46:49.632892-output.json'


In [2]:
with open(coords_path, 'r') as fid:
    coords_data = json.load(fid)

with open(data_path, 'r') as fid:
    route_data = json.load(fid)

In [3]:
route_data.keys()

dict_keys(['slugs', 'non_sucessful_slugs', 'arrivals', 'departures'])

In [9]:
import plotly.express as px


def slug_to_city(slug):
    splitted_slug = slug.split("-")
    uf = splitted_slug[-1].lower()
    place = " ".join(splitted_slug[0:-1])
    address = f'{place} / {uf}'
    return address


source_slug = 'sao-paulo-sp'
departures = route_data['departures'][source_slug]
source_coord = coords_data['coords'][source_slug]


departure_map = {slug_to_city(k): coords_data['coords'].get(k, None)
                 for k in departures}

departure_map = {k: v for k, v in departure_map.items() if v is not None}


lons = [el[0] for el in departure_map.values()]
lats = [el[1] for el in departure_map.values()]


records = [{"destination": k, 'x': v[0], 'y': v[1],
            'uf': k.split("/ ")[-1]} for k, v in departure_map.items()]
df = pd.DataFrame(records)

fig = px.scatter_geo(df,
                     lon='x',
                      lat='y', 
                     color='uf',
                     hover_name='destination',
                     height=600)

fig.update_traces(marker=dict(size=8, symbol='x'))
fig.update_geos(fitbounds="locations", projection_type='orthographic',
    resolution=50,
    showcoastlines=True, coastlinecolor="RebeccaPurple",
    showocean=True, oceancolor="LightBlue",
    showlakes=True, lakecolor="LightBlue",
    showrivers=True, rivercolor="LightBlue",
        showcountries=True, countrycolor="Black")
fig.update_layout(showlegend=False)

fig.show()
